In [1]:
from random import randint
from scipy.stats import mode

import pandas as pd
import numpy as np
from functools import reduce
from collections import Counter

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

df = pd.read_csv('spam_ham.csv')

In [2]:
def conta_palavras(lista_mensagens, dicionario_palavras):
    
    x = []
    for m in lista_mensagens:
        ocorrencias = {i:0 for i in dicionario_palavras}

        for p in m:
            if p in ocorrencias:
                ocorrencias[p] += 1

        x.append(ocorrencias)
        
    return pd.DataFrame(x)
# --------------------------------------------------------------------------------
df.text = df.text.str.lower().str.replace(r'[^\w\s]+', ' ').str.split()

df_treino, df_teste = train_test_split(df, test_size = 0.2)

mensagens = reduce(lambda x, y: x + y, df_treino.text)
palavras_comuns = [i[0] for i in Counter(mensagens).most_common(100)]

x_treino = conta_palavras(df_treino.text, palavras_comuns)
x_teste = conta_palavras(df_teste.text, palavras_comuns)

y_treino = df_treino.type
y_teste = df_teste.type

In [3]:
class RandomClassifier:

    def __init__(self, n, algorithm, models = None, votes = None):
        
        self.n = n
        self.algorithm = algorithm
        self.models = models
        self.votes = votes

    def __repr__(self):
        return f'Random Classifier com {self.n} modelos do tipo {self.algorithm}'
    
    def fit(self, x, y):
        
        exec(f"self.models = [{self.algorithm}(random_state = randint(1, 1000)) for n in range(0, {self.n})]")
        self.models = [model.fit(x, y) for model in self.models] 
            
    def predict(self, x):
        self.votes = [mdl.predict(x) for mdl in self.models]
        return mode(self.votes)[0][0][0]

In [4]:
rc = RandomClassifier(3, 'SVC')

In [5]:
rc

Random Classifier com 3 modelos do tipo SVC

In [6]:
rc.fit(x_treino, y_treino)

In [7]:
rc.predict(x_teste.iloc[[0]])

'ham'

In [8]:
rc.models

[SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
     max_iter=-1, probability=False, random_state=656, shrinking=True, tol=0.001,
     verbose=False),
 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
     max_iter=-1, probability=False, random_state=486, shrinking=True, tol=0.001,
     verbose=False),
 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
     max_iter=-1, probability=False, random_state=154, shrinking=True, tol=0.001,
     verbose=False)]

In [9]:
rc.votes

[array(['ham'], dtype=object),
 array(['ham'], dtype=object),
 array(['ham'], dtype=object)]